##### 问题:
我们想定义各种各样的数据结构，但是对于某些特定的属性，我们想对允许赋给它们
的值强制添加一些限制。

##### 解决方案:
在这个问题中，基本上我们面对的任务就是在设定特定的实例属性时添加检查或者
断言。为了做到这点，需要对每个属性的设定做定制化处理，因此应该使用描述符来
完成。

下面的代码使用描述符实现了一个类型系统以及对值进行检查的框架：

In [20]:
# Base class. Uses a descriptor to set a value
class Descriptor:
    def __init__(self, name=None, **opts):
        self.name = name
        for key, value in opts.items():
            setattr(self, key, value)
    def __set__(self, instance, value):
        instance.__dict__[self.name] = value

# Descriptor for enforcing types
class Typed(Descriptor):
    expected_type = type(None) 
    def __set__(self, instance, value):
        if not isinstance(value, self.expected_type):
            raise TypeError('expected ' + str(self.expected_type))
        super().__set__(instance, value)

# Descriptor for enforcing values
class Unsigned(Descriptor):
    def __set__(self, instance, value):
        if value < 0:
            raise ValueError('Expected >= 0')
        super().__set__(instance, value)
        
class MaxSized(Descriptor):
    def __init__(self, name=None, **opts):
        if 'size' not in opts:   #opts是个字典!size是key,判断是否传入了size的大小
            raise TypeError('missing size option')
        super().__init__(name, **opts)
    def __set__(self, instance, value): #set方法
        if len(value) >= self.size:
            raise ValueError('size must be < ' + str(self.size))
        super().__set__(instance, value)

这些类可作为构建一个数据模型或者类型系统的基础组件。让我们继续，下面这些代
码实现了一些不同类型的数据：

In [21]:
class Integer(Typed):
    expected_type = int

class UnsignedInteger(Integer, Unsigned):
     pass

class Float(Typed):
    expected_type = float

class UnsignedFloat(Float, Unsigned):
    pass

class String(Typed):
    expected_type = str
    
class SizedString(String, MaxSized):
    pass 

有了这些类型对象，现在就可以像这样定义一个类了：

In [22]:
class Stock:
    # Specify constraints
    name = SizedString('name',size=8)
    shares = UnsignedInteger('shares')
    price = UnsignedFloat('price')
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price

有了这些约束后，就会发现现在对属性进行赋值是会进行验证的。示例如下：

In [23]:
s = Stock('ACME', 50, 91.1)
print( s.name )
s.shares = 75
# s.shares = -10

ACME


可以运用一些技术来简化在类中设定约束的步骤。一种方法是使用类装饰器，示例
如下：

In [24]:
# Class decorator to apply constraints
def check_attributes(**kwargs):
    def decorate(cls):
        for key, value in kwargs.items():
            if isinstance(value, Descriptor):
                value.name = key
                setattr(cls, key, value)
            else:
                setattr(cls, key, value(key))
                return cls
    return decorate
# Example
@check_attributes(name=SizedString(size=8),
shares=UnsignedInteger,
price=UnsignedFloat)
class Stock:
 def __init__(self, name, shares, price):
    self.name = name
    self.shares = shares
    self.price = price


另一种方法是使用元类，示例如下：

In [25]:

# A metaclass that applies checking
class checkedmeta(type):
    def __new__(cls, clsname, bases, methods):
    # Attach attribute names to the descriptors
        for key, value in methods.items():
            if isinstance(value, Descriptor):
                value.name = key
        return type.__new__(cls, clsname, bases, methods)
# Example
class Stock(metaclass=checkedmeta):
    name = SizedString(size=8)
    shares = UnsignedInteger()
    price = UnsignedFloat()
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price 

本节涉及了好几种高级技术，包括描述符、mixin 类、对 super()的使用、类装饰器以及
元类。在这里涵盖所有这些主题的基础知识显然是不现实的，读者可以在其他章节中
找到相关的示例（参阅 8.9、8.18、9.12 以及 9.19 节）。但是，还是有几个微妙之处值得我们讨论。

首先，在 Descriptor 基类中会发现有一个__set__()方法，但是却没有与之对应的__get__()
方法。如果一个描述符所做的仅仅只是从底层的实例字典中提取出具有相同名称的
值，那么定义__get__()就是不必要的了。实际上，在这里定义__get__()反而会让程序运
行得更慢。因此，本节只会把重点放在对__set__()的实现上。

本节中各个描述符类的总体设计是基于 mixin 类的。例如，Unsigned 和 MaxSized 类是
用来和其他从 Typed 类中继承而来的描述符类混合在一起使用的。要处理某种特定的
数据类型，我们使用多重继承来将所需要的功能联合在一起使用。

我们也会注意到所有描述符的__init__()方法已经被编写为具有相同的签名形式，其中
涉及关键字参数**opts。MaxSized 类会在 opts 中寻找它所需要的属性，但是会将其传
递给基类 Descriptor，然后在基类中完成实际的设定。像这样的组合类（尤其是 mixin），
一个棘手的地方在于我们并非总是知道这些类是如何串联起来的，或者 super()到底会
调用些什么。基于这个原因，需要保证让所有可能出现的组合类都能正常工作。

各种类型类（type classs）的定义比如 Integer、Float 以及 String 展示了一项有用的技术，
即，使用类变量来定制化实现。描述符 Typed 仅仅是寻找一个 expected_type 属性，该
属性是由那些子类所提供的。

使用类装饰器或者元类常常可以简化用户代码。我们会发现在这些例子中，用户不再
需要多次输入属性名了。示例如下：

In [26]:
# Normal
class Point:
    x = Integer('x')
    y = Integer('y')
# Metaclass
class Point(metaclass=checkedmeta):
    x = Integer()
    y = Integer()

实现类装饰器和元类的代码会扫描类字典，寻找描述符。当找到描述符后，它们会根
据键的值自动填入描述符的名称。

在所有方法中，类装饰器可以提供最大的灵活性和稳健性。第一，这种解决方案不
依赖于任何高级的机制，比如说元类。第二，装饰器可以很容易地根据需要在类定
义上添加或者移除。例如，在装饰器中，可以有一个选项来简单地忽略掉添加的
检查机制。这样就能让检查机制可以根据需要随意打开或关闭（调试环境对比生产
环境）。

最后，采用类装饰器的解决方案也可以用来取代 mixin 类、多重继承以及对 super()函数的使用。下面就是使用类装饰器的备选方案：


In [ ]:
# Base class. Uses a descriptor to set a value
class Descriptor:
    def __init__(self, name=None, **opts):
        self.name = name
        for key, value in opts.items():
            setattr(self, key, value)

    def __set__(self, instance, value):
        instance.__dict__[self.name] = value

    # Decorator for applying type checking
    def Typed(expected_type, cls=None):
        if cls is None:
            return lambda cls: Typed(expected_type, cls)
        super_set = cls.__set__
        def __set__(self, instance, value):
            if not isinstance(value, expected_type):
                raise TypeError('expected ' + str(expected_type))
            super_set(self, instance, value)
        cls.__set__ = __set__
        return cls

# Decorator for unsigned values
def Unsigned(cls):
    super_set = cls.__set__

    def __set__(self, instance, value):
        if value < 0:
            raise ValueError('Expected >= 0')
        super_set(self, instance, value)
    cls.__set__ = __set__
    return cls

# Decorator for allowing sized values
def MaxSized(cls):
    super_init = cls.__init__

    def __init__(self, name=None, **opts):
        if 'size' not in opts:
            raise TypeError('missing size option')
        super_init(self, name, **opts)
    cls.__init__ = __init__
    super_set = cls.__set__

    def __set__(self, instance, value):
        if len(value) >= self.size:
            raise ValueError('size must be < ' + str(self.size))
        super_set(self, instance, value)
    cls.__set__ = __set__
    return cls

# Specialized descriptors
@Typed(int)
class Integer(Descriptor):
    pass

@Unsigned
class UnsignedInteger(Integer):
    pass

@Typed(float)
class Float(Descriptor):
    pass

@Unsigned
class UnsignedFloat(Float):
    pass

@Typed(str)
class String(Descriptor):
    pass

@MaxSized
class SizedString(String):
    pass 

在这个备选方案中定义的类能够像之前那样以完全相同的方式工作（之前的示例代码
都不改变），只是每个部分都会比以前运行得更快。例如，对设定一个类型属性做简单
的计时测试就能发现，采用类装饰器的方案运行速度要比采用 mixin 类的方案几乎快
上 100%。读到这里你难道还会不开心吗？